Нулевая гипотеза (H0): Изменение положения и цвета кнопки НЕ приведет к увеличению конверсии.  

Альтернативная гипотеза (H1): Изменение положения и цвета кнопки приведет к увеличению конверсии. Пользователи группы В будут чаще открывать контакты.

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import seaborn as sns
from math import ceil
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

Найдем базовую конверсию на основании данных с 1 ноября по 9 ноября

In [4]:
df = pd.read_csv('exp1.csv') #получим датафрейм из файла

In [5]:
bc_df = df.query("date < '2017-11-10'") #фильтруем по дате
bc_df

,user_id,date,url
0,d48e135cA,2017-11-01 00:00:00,/dummy.txt?action=viewbull_open_photo_fotorama
1,7885b94eB,2017-11-01 00:00:00,/city/directory/57689556.html
2,9382d05eB,2017-11-01 00:00:01,/directory/?_lightweight=1&_origuterm=abond-fr...
3,feaaa515A,2017-11-01 00:00:01,/dummy.txt?action=mobileapp_native_banner_show
4,feaaa515A,2017-11-01 00:00:01,/dummy.txt?action=mobileVersion
...,...,...,...
1206141,4e9a98b4A,2017-11-09 23:59:56,/dummy.txt?action=viewdir_show_plate&keyName=d...
1206142,73163a57B,2017-11-09 23:59:56,/dummy.txt?action=page_clicked&keyName=8
1206143,3e9db1dfA,2017-11-09 23:59:58,/dummy.txt?action=adding__v2__field_changed&ke...
1206144,4e9a98b4A,2017-11-09 23:59:58,/dummy.txt?action=viewdir_show_plate&keyName=f...


Отберем только те посещения, где пользователи открывали карточку с 1 по 9 ноября

In [7]:
view_card = bc_df[bc_df['url'].str.contains(r"/city/directory/\d+\.html")]
print(view_card)

           user_id                 date                            url
1        7885b94eB  2017-11-01 00:00:00  /city/directory/57689556.html
29       7885b94eB  2017-11-01 00:00:27  /city/directory/58425632.html
43       9382d05eB  2017-11-01 00:00:37  /city/directory/43376524.html
65       5ec519baA  2017-11-01 00:00:53  /city/directory/53543073.html
76       ebdb2e66B  2017-11-01 00:01:02  /city/directory/47388240.html
...            ...                  ...                            ...
1206075  4e9a98b4A  2017-11-09 23:58:59  /city/directory/53656353.html
1206079  e1dd36f4B  2017-11-09 23:59:02  /city/directory/58426566.html
1206106  5e841aadA  2017-11-09 23:59:26  /city/directory/58034184.html
1206124  9431afd9B  2017-11-09 23:59:46  /city/directory/51741546.html
1206132  8c897633A  2017-11-09 23:59:50  /city/directory/50246077.html

[122742 rows x 3 columns]


Количество 
пользователей, посетивших карточку с 1 по 9 ноября

In [9]:
# users_view_card = view_card['user_id'].nunique()
users_view_card = len(view_card['user_id'])
print(users_view_card)

122742


Количество пользователей, посетивших контакт с 1 по 9 ноября

In [11]:
view_contact = bc_df[bc_df['url'].str.contains(r"/good/\d+/owner_info|/bulletin/\d+/ajax_contact")]

In [12]:
# users_view_contact = view_contact['user_id'].nunique()
users_view_contact = len(view_contact['user_id'])
print(users_view_contact)

5223


In [13]:
conversion = users_view_contact / users_view_card

print(f'{users_view_card} пользователей видели карточки, {users_view_contact} открыли контакты, конверсия: {conversion:.2%}')

122742 пользователей видели карточки, 5223 открыли контакты, конверсия: 4.26%


На основании полученных данных будем опираться на следующие значения:

Базовый коэффициент конверсии - 4.26%

Минимальный обнаруживаемый эффект - 1%

Статистическая значимость - 5%

Проведем А/А тестирование

In [16]:
df['group'] = df.user_id.apply(lambda x: 'A' if x[-1] == 'A' else 'B') #добавляем столбец группы
df['converted'] = df.url.str.contains(r"/good/\d+/owner_info|/bulletin/\d+/ajax_contact").astype(int) #добавляем столбец взаимодействия с контактом

In [17]:
filtered_df = df[(df.url.str.contains(r"/city/directory/\d+\.html")) | (df['converted'] == 1)]

In [18]:
aa_df = filtered_df.query("date < '2017-11-10'")

In [19]:
control_sample = aa_df[aa_df.group == 'A']
treatment_sample = aa_df[aa_df.group == 'B']

In [20]:
aa_test = pd.concat([control_sample, treatment_sample], axis=0)
aa_test.reset_index(drop=True, inplace = True)

In [21]:
def std_dev(x):
    return np.std(x, ddof=0)
def std_error(x):
    return stats.sem(x, ddof=0)

In [22]:
conversion_rate = aa_test.groupby('group').converted.agg(['mean', std_dev, std_error])
conversion_rate.columns = ['conversion_rate','std_deviation','std_error']
conversion_rate

,conversion_rate,std_deviation,std_error
group,,,
A,0.038970,0.193524,0.000739
B,0.042948,0.202740,0.000832


In [23]:
control_results = aa_test[aa_test.group == 'A'].converted
num_control = control_results.count()

treatment_results = aa_test[aa_test.group == 'B' ].converted
num_treatment = treatment_results.count()

successes = [control_results.sum(), treatment_results.sum()]
nobs = [num_control, num_treatment]

In [24]:
z_stat, pval = proportions_ztest(successes, nobs = nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs)

In [25]:
print(f'Z-статистика: {z_stat:.2}')
print(f'P-value: {pval:.3}')
print(f'Доверительный интервал для доверительной вероятности 95% в группе A [{lower_con:.3},{upper_con:.3}]')
print(f'Доверительный интервал для доверительной вероятности 95% в группе B [{lower_treat:.3},{upper_treat:.3}]')

Z-статистика: -3.6
P-value: 0.000335
Доверительный интервал для доверительной вероятности 95% в группе A [0.0375,0.0404]
Доверительный интервал для доверительной вероятности 95% в группе B [0.0413,0.0446]


Значение p-value значительно меньше уровня статистической значимости 5% (0.05), что говорит о статистически значимой разнице между группами A и B, даже при условии, что это A/A тест.

Обычно в A/A тестировании разницы не ожидается, так как обе группы должны демонстрировать одинаковые показатели. Здесь же мы видим статистически значимую разницу, что может указывать на проблему с экспериментальным дизайном, например, систематические ошибки, нарушение случайного распределения, ошибки в данных или непредвиденные факторы, влияющие на результат.

Проведем А/B тестирование

Ограничим датасет для отображения данных с 10 ноября по 1 декабря включительно

In [29]:
ab_df = filtered_df.query("'2017-11-10' <= date < '2017-12-02'")

In [30]:
ab_df

,user_id,date,url,group,converted
1206146,c4722e52A,2017-11-10 00:00:02,/city/directory/54442679.html,A,0
1206147,d889038dA,2017-11-10 00:00:03,/city/directory/46307421.html,A,0
1206163,f0fd895cB,2017-11-10 00:00:26,/city/directory/57179476.html,B,0
1206169,d555480A,2017-11-10 00:00:29,/city/directory/57131892.html,A,0
1206178,fbbdabb6A,2017-11-10 00:00:46,/city/directory/52350033.html,A,0
...,...,...,...,...,...
4161537,1be0d3fbA,2017-12-01 23:59:48,/city/directory/58498802.html,A,0
4161543,3096b098B,2017-12-01 23:59:50,/city/directory/55928827.html,B,0
4161553,c38dd6cbB,2017-12-01 23:59:53,/city/directory/51768927.html,B,0
4161558,3540df26A,2017-12-01 23:59:53,/city/directory/42798560.html,A,0


In [31]:
control_sample = ab_df[ab_df.group == 'A']
treatment_sample = ab_df[ab_df.group == 'B']

In [32]:
ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace = True)

In [33]:
conversion_rate = ab_test.groupby('group').converted.agg(['mean', std_dev, std_error])
conversion_rate.columns =['conversion_rate','std_deviation','std_error']
conversion_rate

,conversion_rate,std_deviation,std_error
group,,,
A,0.042797,0.202400,0.000489
B,0.044526,0.206262,0.000534


In [34]:
control_results = ab_test[ab_test.group =='A'].converted
num_control = control_results.count()

treatment_results = ab_test[ab_test.group =='B'].converted
num_treatment = treatment_results.count()

successes = [control_results.sum(), treatment_results.sum()]
nobs = [num_control, num_treatment]

In [36]:
z_stat, pval = proportions_ztest(successes, nobs = nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs)

In [37]:
print(f'Z-статистика: {z_stat:.2}')
print(f'P-value: {pval:.3}')
print(f'Доверительный интервал для доверительной вероятности 95% в группе A [{lower_con:.3},{upper_con:.3}]')
print(f'Доверительный интервал для доверительной вероятности 95% в группе B [{lower_treat:.3},{upper_treat:.3}]')

Z-статистика: -2.4
P-value: 0.0168
Доверительный интервал для доверительной вероятности 95% в группе A [0.0418,0.0438]
Доверительный интервал для доверительной вероятности 95% в группе B [0.0435,0.0456]


Значение p-value составляет 0.0168, что также меньше 0.05, указывая на статистически значимую разницу между группой A и группой B.

Доверительные интервалы частично перекрываются, что указывает на то, что разница между конверсиями групп менее однозначна. Это создает сомнения в статистической значимости результата, поскольку пересечение доверительных интервалов может означать, что наблюдаемая разница обусловлена случайными колебаниями.